# Socket Worker Client Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled SocketWorker Server.ipynb and is in the same folder as this notebook. You should execute this notebook AFTER you have executed the other one.

In this tutorial, we'll demonstrate how to launch a SocketWorker server which will listen for PyTorch commands over a socket connection. This tutorial is a followup to the VirtualWorker tutorial (https://github.com/OpenMined/PySyft/blob/master/examples/Remote%20PyTorch%20using%20Virtual%20Worker.ipynb) where the only difference is that in this tutorial the two workers are connected via a socket connection on the localhost network.

If you'd like to circumvent the need to install dependencies, this notebook is also available as as Colab notebook

*Colab:* https://colab.research.google.com/drive/1Je1rk7olA9uTWWaqvvt4_gXf7yX1rTBm

# Step 1: Hook Torch and Create Local Worker

In this step, we hook PyTorch and initialize within the hook a client SocketWorker.

In [1]:
import syft as sy

hook = sy.TorchHook(local_worker=sy.SocketWorker(id=0, port=1111))

Starting Socket Worker...
Ready!


# Step 2: Create Pointer to Remote Socket Worker

In order to interact with a foreign worker over a socket connection, we need to create a pointer to it containing information on how to contact it. We set the is_pointer=True to signify that this Python object is not in fact a worker in and of itself but that it is merely a pointer to one over the network. We then inform our local worker about this pointer.

In [2]:
remote_client = sy.SocketWorker(hook=hook,id=2, port=1131, is_pointer=True)
hook.local_worker.add_worker(remote_client)

Attaching Pointer to Socket Worker...


# Step 3: Create Tensors & Send To The Worker

In [3]:
x = sy.FloatTensor([1,2,3,4,5]).send(remote_client)

In [4]:
x2 = sy.FloatTensor([1,2,3,4,4]).send(remote_client)

In [5]:
x

FloatTensor[_PointerTensor - id:9355972172 owner:0 loc:2 id@loc:61633545981]

In [6]:
x2

FloatTensor[_PointerTensor - id:8666866360 owner:0 loc:2 id@loc:89765971380]

# Step 4: Execute Operations Like Normal

In [7]:
y = x + x2 + x

# Step 5: Get Results

In [8]:
y

FloatTensor[_PointerTensor - id:7366182616 owner:0 loc:2 id@loc:4693269260]

In [9]:
y.owner

<syft.core.workers.socket.SocketWorker id:0>

In [10]:
y.get()


  3
  6
  9
 12
 14
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [11]:
y


  3
  6
  9
 12
 14
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

# Step 6: Search For Tensors On Remote Machine

In [12]:
pointers = remote_client.search("#boston_housing")

b'\x82\xa3obj\x92\x81\xa9__tuple__\x92\x82\xa3obj\x81\xac__Variable__\x84\xaatorch_type\xa8Variable\xa4data\x81\xaf__FloatTensor__\x83\xaatorch_type\xabFloatTensor\xa4data\x90\xa5child\x81\xb0___LocalTensor__\x83\xa5owner\x02\xa2id\xce\x9a\x83\x19*\xaatorch_type\xabFloatTensor\xa5child\x81\xb2___PointerTensor__\x85\xa5owner\x02\xa2id\xcf\x00\x00\x00\t}\x11\xeb\xe4\xa8location\x02\xaeid_at_location\xd9 #boston_housing #target #dataset\xaatorch_type\xa8Variable\xadrequires_grad\xc2\xa4mode\xa7acquire\x91\x81\xb2___PointerTensor__\x85\xa5owner\x02\xa2id\xcf\x00\x00\x00\t}\x11\xeb\xe4\xa8location\x02\xaeid_at_location\xd9 #boston_housing #target #dataset\xaatorch_type\xa8Variable\x81\xa9__tuple__\x92\x82\xa3obj\x81\xac__Variable__\x84\xaatorch_type\xa8Variable\xa4data\x81\xaf__FloatTensor__\x83\xaatorch_type\xabFloatTensor\xa4data\x90\xa5child\x81\xb0___LocalTensor__\x83\xa5owner\x02\xa2id\xce\x8a\xed\x94\xa2\xaatorch_type\xabFloatTensor\xa5child\x81\xb2___PointerTensor__\x85\xa5owner\x02\

TypeError: byte indices must be integers or slices, not str

In [13]:
pointers

[Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension],
 Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension]]

In [14]:
x = pointers[0]

In [15]:
y = x + x

In [16]:
x.get()

Variable containing:
-2
-1
 0
 1
 2
 3
[syft.core.frameworks.torch.tensor.FloatTensor of size 6]

In [17]:
y.get()

Variable containing:
-4
-2
 0
 2
 4
 6
[syft.core.frameworks.torch.tensor.FloatTensor of size 6]

In [18]:
# you can also search on multiple conditions at a time
pointers = remote_client.search(["#nashville_housing", "#input"])

In [19]:
pointers

[Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension]]

In [20]:
# you can also search on multiple conditions at a time
pointers = remote_client.search(["#"])

In [21]:
pointers # notice that there are 3 since we called .get() on one of them (there were originally 4)

[Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension],
 Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension],
 Variable containing:[syft.core.frameworks.torch.tensor.FloatTensor with no dimension]]